In [ ]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
# Function to write data to CSV file

def write_to_csv(data, filename):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write header row
        writer.writerow(['title', 'link', 'date', 'content', 'is_fake', 'media_bias'])
        # Write data rows
        for row in data:
            writer.writerow(row)

In [ ]:
# Tempo Fake News Scraping

url = 'https://cekfakta.tempo.co/2020/' # for year 2020

# Initialize list to store data
data = []

for i in range(1, 13):
  if (i < 10):
    url_page = url+'0{}'.format(i)
  else:
    url_page = url+'{}'.format(i)
  page = requests.get(url_page)
  soup = BeautifulSoup(page.text, 'html')

  print(i)

  # Find all news articles
  news = soup.find_all('article', 'text-card')

  for n in news:
    title = n.find('a').get_text()
    link = n.find('a')['href']

    # Split title and category
    title_first_space_index = title.find(',')
    category = title[:title_first_space_index]
    title_news = title[title_first_space_index + 1:].strip()

    if (category == "Menyesatkan" or category == "Keliru") and link is not None:
      # Get news contents
      if link.startswith("//"):
        news_link = "http:" + link
      else:
        news_link = link
      get_content = requests.get(news_link)
      soup_content = BeautifulSoup(get_content.text, 'html')
      date = soup_content.find('div', 'detail-title').find('p', 'date margin-bottom-sm').get_text()
      content = soup_content.find('div', 'detail-in').find_all('p')
      paragraph = ''
      for p in content:
        #p.span.unwrap()
        paragraph += p.get_text()

      # News information source
      is_fake = 1
      media_bias = 'left'

      # Append data to list
      data.append([title_news, news_link, date, paragraph, is_fake, media_bias])

# File name for the CSV
filename = 'dataset_tempo_hoaks_draft.csv'

# Write data to CSV file
write_to_csv(data, filename)

print(f"Data has been written to {filename}")

In [ ]:
# Read CSV file
df = pd.read_csv('/content/dataset_tempo_hoaks_draft.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734 entries, 0 to 733
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       734 non-null    object
 1   link        734 non-null    object
 2   date        734 non-null    object
 3   content     733 non-null    object
 4   is_fake     734 non-null    object
 5   media_bias  734 non-null    object
dtypes: object(6)
memory usage: 34.5+ KB


,title,link,date,content,is_fake,media_bias
0,Video Praktik Penyebaran Racun Melalui Awan Bu...,https://cekfakta.tempo.co/fakta/2931/keliru-vi...,"Sabtu, 1 Juni 2024 13:35 WIB",Video berdurasi 34 detik dengan klaim praktik ...,1,left
1,Klaim bahwa Disease X Bocoran Pandemi Berikutn...,https://cekfakta.tempo.co/fakta/2930/keliru-kl...,"Sabtu, 1 Juni 2024 13:04 WIB",Sebuah akun Instagram [arsip] mengunggah cupli...,1,left
2,Virus SARS-CoV-2 Telah Dipatenkan Sejak 2015,https://cekfakta.tempo.co/fakta/2929/keliru-vi...,"Sabtu, 1 Juni 2024 12:01 WIB",Tangkapan layar sebuah jurnal penelitian berju...,1,left
3,Konten Pernyataan Peneliti BRIN Syafuan Rozy T...,https://cekfakta.tempo.co/fakta/2928/menyesatk...,"Kamis, 30 Mei 2024 15:28 WIB",Dalam sebulan terakhir tagar #Tumpas OPM atau ...,1,left
4,Konten yang Berisi Klaim Kematian Mendadak yan...,https://cekfakta.tempo.co/fakta/2925/keliru-ko...,"Rabu, 29 Mei 2024 14:21 WIB",Sebuah unggahan video pendek berisi klaim tent...,1,left


In [ ]:
# remove duplicate data
df_unique = df.drop_duplicates(subset='title', keep='first')

# save new data
df_unique.to_csv('dataset_tempo_hoaks_2020.csv', index=False)

df_unique.info()
df_unique.head()

<class 'pandas.core.frame.DataFrame'>
Index: 358 entries, 0 to 725
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       358 non-null    object
 1   link        358 non-null    object
 2   date        358 non-null    object
 3   content     357 non-null    object
 4   is_fake     358 non-null    object
 5   media_bias  358 non-null    object
dtypes: object(6)
memory usage: 19.6+ KB


,title,link,date,content,is_fake,media_bias
0,Video Praktik Penyebaran Racun Melalui Awan Bu...,https://cekfakta.tempo.co/fakta/2931/keliru-vi...,"Sabtu, 1 Juni 2024 13:35 WIB",Video berdurasi 34 detik dengan klaim praktik ...,1,left
1,Klaim bahwa Disease X Bocoran Pandemi Berikutn...,https://cekfakta.tempo.co/fakta/2930/keliru-kl...,"Sabtu, 1 Juni 2024 13:04 WIB",Sebuah akun Instagram [arsip] mengunggah cupli...,1,left
2,Virus SARS-CoV-2 Telah Dipatenkan Sejak 2015,https://cekfakta.tempo.co/fakta/2929/keliru-vi...,"Sabtu, 1 Juni 2024 12:01 WIB",Tangkapan layar sebuah jurnal penelitian berju...,1,left
3,Konten Pernyataan Peneliti BRIN Syafuan Rozy T...,https://cekfakta.tempo.co/fakta/2928/menyesatk...,"Kamis, 30 Mei 2024 15:28 WIB",Dalam sebulan terakhir tagar #Tumpas OPM atau ...,1,left
4,Konten yang Berisi Klaim Kematian Mendadak yan...,https://cekfakta.tempo.co/fakta/2925/keliru-ko...,"Rabu, 29 Mei 2024 14:21 WIB",Sebuah unggahan video pendek berisi klaim tent...,1,left


In [ ]:
# Save dataset to Google Drive

from google.colab import drive
import shutil

drive.mount('/content/drive')

colab_file_path = '/content/dataset_tempo_hoaks_2020.csv'
drive_file_path = '/content/drive/My Drive/Colab Notebooks/Bangkit | PukulRata/dataset_tempo_hoaks_2020.csv'

shutil.copyfile(colab_file_path, drive_file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/Colab Notebooks/Bangkit | PukulRata/dataset_tempo_hoaks_2020.csv'

In [ ]:
# merge tempo hoax dataset
ds_2020 = "dataset_tempo_hoaks_2020.csv"
ds_2021 = "dataset_tempo_hoaks_2021.csv"
ds_2022 = "dataset_tempo_hoaks_2022.csv"
ds_2023 = "dataset_tempo_hoaks_2023.csv"
ds_2024 = "dataset_tempo_hoaks_2024.csv"

# List of CSV file names to merge
csv_files = [ds_2020, ds_2021, ds_2022, ds_2023, ds_2024]

# Reads each CSV file and saves it in a list
data_frames = [pd.read_csv(file) for file in csv_files]

# Combines all DataFrames in the list into one DataFrame
combined_df = pd.concat(data_frames, ignore_index=True)

# remove duplicate title content
combined_df = combined_df.drop_duplicates(subset=['title'], keep='first')

# Saves the merged DataFrame into a new CSV file
combined_df.to_csv('dataset_tempo_hoaks_20_24.csv', index=False)

print("CSV files merged successfully!")

In [ ]:
# Read CSV file
df = pd.read_csv("dataset_tempo_hoaks_20_24.csv")
df.info()
df.head()

In [ ]:
# Save dataset to Google Drive

from google.colab import drive
import shutil

drive.mount('/content/drive')

colab_file_path = '/content/dataset_tempo_hoaks_20_24.csv'
drive_file_path = '/content/drive/My Drive/Colab Notebooks/Bangkit | PukulRata/dataset_tempo_hoaks_20_24.csv'

shutil.copyfile(colab_file_path, drive_file_path)